## Librerias 

In [24]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gdp
import shapely
import json
from bson.json_util import dumps
from pandas import DataFrame

## Importacion de datos 

In [61]:
dft = pd.read_csv('../data/Tuenti.csv')
dft

,enterprise,team,requirements,name,latitud,longitud
0,Tuenti Technologies,Executives,starbucks,Starbucks,40.415323,-3.694949
1,Tuenti Technologies,Executives,starbucks,Starbucks,40.418038,-3.699372
2,Tuenti Technologies,Executives,starbucks,Starbucks,40.420752,-3.699315
3,Tuenti Technologies,Executives,starbucks,Starbucks,40.420140,-3.702886
4,Tuenti Technologies,Executives,starbucks,Starbucks,40.419668,-3.706135
...,...,...,...,...,...,...
93,Tuenti Technologies,Designers,Museum,La Casa Del Ratoncito Perez,40.416947,-3.705552
94,Tuenti Technologies,Designers,Museum,Fundacion COAM,40.423158,-3.694643
95,Tuenti Technologies,Designers,Museum,Guernica By Pablo Picasso,40.408134,-3.694328
96,Tuenti Technologies,Designers,Museum,La Fábrica,40.426295,-3.693019


In [63]:
dffon = pd.read_csv('../data/Fon.csv')
dffon

,enterprise,team,requirements,name,latitud,longitud
0,Fon,Executives,starbucks,Starbucks,40.415323,-3.694949
1,Fon,Executives,starbucks,Starbucks,40.418038,-3.699372
2,Fon,Executives,starbucks,Starbucks,40.420752,-3.699315
3,Fon,Executives,starbucks,Starbucks,40.420140,-3.702886
4,Fon,Executives,starbucks,Starbucks,40.419668,-3.706135
...,...,...,...,...,...,...
93,Fon,Designers,Museum,La Casa Del Ratoncito Perez,40.416947,-3.705552
94,Fon,Designers,Museum,Fundacion COAM,40.423158,-3.694643
95,Fon,Designers,Museum,Guernica By Pablo Picasso,40.408134,-3.694328
96,Fon,Designers,Museum,La Fábrica,40.426295,-3.693019


## GeoData

Tratamiento de las latitudes y las longitudes de las bases de datos iniciales para su  manipulacion en mongo, obteniendo asi las distancias relativas a las distintas localizaciones respecto a los candidatos Tuenti y Fon.

In [86]:
# generacion de bbdd y collecion 
client = MongoClient()
db = client.geoloca # creamos bbdd
collection = db.tuenti # creamos collecion para tuenti
collection = db.fon # creamos collecion para fon

#####  Tuenti

In [64]:
# generamos geometry que es el geojson

dft = gdp.GeoDataFrame(dft, geometry= gdp.points_from_xy(dft.longitud, dft.latitud ))
dft.columns=['enterprise', 'team','requirements','name','latitud','longitud','geometry' ]
    
#Lo aplicamos a toda la columna
dft['geometry']= dft['geometry'].apply(lambda x:shapely.geometry.mapping(x))

/usr/local/Caskroom/miniconda/base/envs/ironhack=3.8/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [66]:
# creamos index pasando una clave para cada objeto geoespacial y pasamos 2dspher para que lo entienda mongo 
collection.create_index([("geometry", "2dsphere")]) 

'geometry_2dsphere'

In [67]:
datat = dft.to_dict(orient='records')
datat

[{'enterprise': 'Tuenti Technologies',
  'team': 'Executives',
  'requirements': 'starbucks',
  'name': 'Starbucks',
  'latitud': 40.41532327347708,
  'longitud': -3.694948668774415,
  'geometry': {'type': 'Point',
   'coordinates': (-3.694948668774415, 40.41532327347708)}},
 {'enterprise': 'Tuenti Technologies',
  'team': 'Executives',
  'requirements': 'starbucks',
  'name': 'Starbucks',
  'latitud': 40.41803818519477,
  'longitud': -3.699371513099982,
  'geometry': {'type': 'Point',
   'coordinates': (-3.699371513099982, 40.41803818519477)}},
 {'enterprise': 'Tuenti Technologies',
  'team': 'Executives',
  'requirements': 'starbucks',
  'name': 'Starbucks',
  'latitud': 40.42075183549848,
  'longitud': -3.699314529721079,
  'geometry': {'type': 'Point',
   'coordinates': (-3.699314529721079, 40.42075183549848)}},
 {'enterprise': 'Tuenti Technologies',
  'team': 'Executives',
  'requirements': 'starbucks',
  'name': 'Starbucks',
  'latitud': 40.42014039388448,
  'longitud': -3.702886

In [68]:
collection.insert_many(datat)

calculamos distancias haciendo una agragacion mediante $geoNear 

In [82]:
  ########## # para tuenti 40.415824	-3.696277 # ###########  	    
fillter =  [{ "$geoNear": {'near': [-3.696277, 40.415824],
                 'distanceField': 'distance',
                 'maxDistance': 1000,
                 'distanceMultiplier': 6371, 
                 'spherical'  : True}}]
distance = collection.aggregate(fillter)
tuenti_result = json.loads(dumps(distance))

In [108]:
distance_tuenti = pd.DataFrame(list(tuenti_result))

#####  Fon

In [87]:
dffon = gdp.GeoDataFrame(dffon, geometry= gdp.points_from_xy(dffon.longitud, dffon.latitud ))
dffon.columns=['enterprise', 'team','requirements','name','latitud','longitud','geometry' ]    
dffon['geometry']= dffon['geometry'].apply(lambda x:shapely.geometry.mapping(x))

/usr/local/Caskroom/miniconda/base/envs/ironhack=3.8/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [88]:
collection.create_index([("geometry", "2dsphere")]) 

'geometry_2dsphere'

In [90]:
dataf = dffon.to_dict(orient='records')

In [91]:
collection.insert_many(dataf)

In [92]:
  ########## #  Fon 40.530724	-3.632687 # ###########  	   
fillter =  [{ "$geoNear": {'near': [-3.632687, 40.530724],
                 'distanceField': 'distance',
                 'maxDistance': 7000,
                 'distanceMultiplier': 6371, 
                 'spherical'  : True}}]
distance = collection.aggregate(fillter)
fon_result = json.loads(dumps(distance))

In [93]:
distance_fon = pd.DataFrame(list(fon_result))

Una vez obtenidas las distancias podemos comparar los distintos requerimiento de los  departamentos de la empresa.  

In [125]:
recap = pd.concat([distance_tuenti,distance_fon])
recap[:3]

,_id,enterprise,team,requirements,name,latitud,longitud,geometry,distance
0,{'$oid': '6029b151683fd4a212ef83ba'},Tuenti Technologies,Designers,Museum,Exposición Hopper,40.416153,-3.694864,"{'type': 'Point', 'coordinates': [-3.694864176...",0.125091
1,{'$oid': '6029b151683fd4a212ef8370'},Tuenti Technologies,Executives,starbucks,Starbucks,40.415323,-3.694949,"{'type': 'Point', 'coordinates': [-3.694948668...",0.125485
2,{'$oid': '6029b151683fd4a212ef839f'},Tuenti Technologies,Designers,Museum,Museo Thyssen-Bornemisza,40.416359,-3.694891,"{'type': 'Point', 'coordinates': [-3.694890813...",0.131585


In [118]:
final = recap.groupby(['enterprise','requirements']).mean()
final

latitud  longitud   distance
enterprise          requirements                                    
Fon                 Airport Terminal  40.406544 -3.682036  14.425427
                    Museum            40.417796 -3.697665  13.732991
                    starbucks         40.425117 -3.696769  13.352604
Tuenti Technologies Airport Terminal  40.406544 -3.682036   1.587001
                    Museum            40.417796 -3.697665   1.022286
                    starbucks         40.425117 -3.696769   3.361391

In [124]:
top = final.sort_values('distance')
top

latitud  longitud   distance
enterprise          requirements                                    
Tuenti Technologies Museum            40.417796 -3.697665   1.022286
                    Airport Terminal  40.406544 -3.682036   1.587001
                    starbucks         40.425117 -3.696769   3.361391
Fon                 starbucks         40.425117 -3.696769  13.352604
                    Museum            40.417796 -3.697665  13.732991
                    Airport Terminal  40.406544 -3.682036  14.425427

Decidimo emplazar las nuevas oficinas en los alrededores de "Tuenti Technologies" ya que la mayoria de las distancias medias de los servicios y localizaciones son alli menores. 